# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [22]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By


# Passo 1: Pegar a cotação do dólar.
    # Abrir o navegador.
navegador = webdriver.Edge()
    
    # Entrar no google.
navegador.get("https://google.com.br/")

    # Pesquisar cotação do dólar no google.
navegador.find_element("xpath", 
                       "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input"
                      ).send_keys("cotação dólar")
navegador.find_element("xpath", 
                       "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input"
                      ).send_keys(Keys.ENTER)
    # Pegar a cotação.
cotacao_dolar = navegador.find_element("xpath", 
                     '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
                                    ).get_attribute("data-value")

    
    # Passo 2: Pegar a cotação do euro.
navegador.get("https://google.com.br/")
navegador.find_element("xpath", 
                       "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input"
                      ).send_keys("cotação euro")
navegador.find_element("xpath", 
                       "/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input"
                      ).send_keys(Keys.ENTER)
    # Pegar a cotação.
cotacao_euro = navegador.find_element("xpath", 
                     '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]'
                                     ).get_attribute("data-value")

# Passo 3: Pegar a cotação do ouro.
navegador.get('https://www.melhorcambio.com/ouro-hoje')

cotacao_ouro = navegador.find_element("xpath", '//*[@id="comercial"]').get_attribute("value").replace(",",".")

# Passo 4: Atualizar a base de dados.
# Passo 5: Recalcular os preços.
# Passo 6: Exportar a base de dados.
print(cotacao_dolar)
print(cotacao_euro)
print(cotacao_ouro)

navegador.quit()

5.2442
5.342712297
292.78


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

- Atualizando os preços e o cálculo do Preço Final

In [31]:
# Passo 4: Atualizar a base de dados com os valores da nova cotação.
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")

tabela.loc[base_dados["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)
tabela.loc[base_dados["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)
tabela.loc[base_dados["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# Passo 5: Recalcular os preços.
    # Preço de compra = Preço original * Cotação
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]
    # Preço de venda = Preço de compra * Margem
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]

display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.244200,5244.147558,1.40,7341.806581
1,Carro Renault,4500.00,Euro,5.342712,24042.205337,2.00,48084.410673
2,Notebook Dell,899.99,Dólar,5.244200,4719.727558,1.70,8023.536849
3,IPhone,799.00,Dólar,5.244200,4190.115800,1.70,7123.196860
4,Carro Fiat,3000.00,Euro,5.342712,16028.136891,1.90,30453.460093
5,Celular Xiaomi,480.48,Dólar,5.244200,2519.733216,2.00,5039.466432
6,Joia 20g,20.00,Ouro,292.780000,5855.600000,1.15,6733.940000


### Agora vamos exportar a nova base de preços atualizada

In [33]:
# Passo 6: Exportar a base de dados.
tabela.to_excel("Produtos Novo.xlsx", index=False)